In [1]:
import pandas as pd
import numpy as np

# Crear un DataFrame con datos ficticios
data = {
    'age': [25, np.nan, 35, 40, 29],
    'income': [50000, 60000, 70000, np.nan, 45000],
    'gender': ['male', 'female', np.nan, 'female', 'male'],
    'purchased': [1, 0, 1, 0, 1]
}

df = pd.DataFrame(data)
print("Datos originales:\n", df)


Datos originales:
     age   income  gender  purchased
0  25.0  50000.0    male          1
1   NaN  60000.0  female          0
2  35.0  70000.0     NaN          1
3  40.0      NaN  female          0
4  29.0  45000.0    male          1


Solución paso a paso
1. Imputación de valores faltantes

Usaremos SimpleImputer de scikit-learn para manejar los valores faltantes:

    Imputaremos la media para las características numéricas.
    Imputaremos la moda para las características categóricas.

2. Codificación de variables categóricas

Convertiremos las variables categóricas a numéricas usando OneHotEncoder.
3. Escalado de características numéricas

Aplicaremos StandardScaler para estandarizar las características numéricas.
4. Pipeline

Uniremos todas las transformaciones en un Pipeline para facilitar su aplicación.

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Separar características y etiquetas
X = df.drop('purchased', axis=1)
y = df['purchased']

# Identificar columnas numéricas y categóricas
numeric_features = ['age', 'income']
categorical_features = ['gender']

# Preprocesamiento para características numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputación con la media
    ('scaler', StandardScaler())                 # Escalado
])

# Preprocesamiento para características categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación con la moda
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # Codificación One-Hot
])

# Combinar transformadores en un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Crear el pipeline completo
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Aplicar el pipeline al dataset
X_processed = pipeline.fit_transform(X)
print("Datos transformados:\n", X_processed.toarray() if hasattr(X_processed, 'toarray') else X_processed)


Datos transformados:
 [[-1.41775817 -0.72777814  0.          1.        ]
 [ 0.          0.43666688  1.          0.        ]
 [ 0.53777034  1.6011119   1.          0.        ]
 [ 1.5155346   0.          1.          0.        ]
 [-0.63554677 -1.31000065  0.          1.        ]]


Resultados

    Imputación de valores faltantes:
        Las edades e ingresos faltantes se reemplazan por la media.
        Los géneros faltantes se reemplazan por la moda ('male' o 'female').

    Codificación de variables categóricas:
        gender se convierte en variables dummy (e.g., gender_male, gender_female).

    Escalado de características numéricas:
        age e income se transforman para tener media 0 y desviación estándar 1.

In [6]:
# Obtener nombres de las columnas transformadas
categorical_names = pipeline.named_steps['preprocessor'] \
    .named_transformers_['cat'] \
    .named_steps['onehot'].get_feature_names_out(categorical_features)

# Combinar nombres numéricos y categóricos
column_names = numeric_features + list(categorical_names)

# Convertir a DataFrame
X_transformed_df = pd.DataFrame(X_processed, columns=column_names)
print("Dataset transformado:\n", X_transformed_df)


Dataset transformado:
         age    income  gender_female  gender_male
0 -1.417758 -0.727778            0.0          1.0
1  0.000000  0.436667            1.0          0.0
2  0.537770  1.601112            1.0          0.0
3  1.515535  0.000000            1.0          0.0
4 -0.635547 -1.310001            0.0          1.0


In [7]:
from sklearn.ensemble import RandomForestClassifier

# Agregar modelo al pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Dividir datos y ajustar el modelo
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipeline.fit(X_train, y_train)

# Evaluar
print("Precisión del modelo:", pipeline.score(X_test, y_test))


Precisión del modelo: 0.5
